In [1]:
import os

In [2]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
from random import randint

import pandas as pd
import numpy as np

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from implicit.als import AlternatingLeastSquares

from pathlib import Path

from lightfm import LightFM

In [5]:
DATA_PATH = Path("..", "data", "kion_train")

In [ ]:
!mkdir ../data
!wget https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip -O ../data/data_original.zip
!unzip ../data/data_original.zip -d ../data
!rm -rf ../data/data_original.zip ../data/__MACOSX

# LOAD DATA

In [6]:
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

In [7]:
YEAR_FROM = 1990
STEP_SIZE = 5
bins = [year for year in range(YEAR_FROM, int(items['release_year'].max()) + STEP_SIZE, STEP_SIZE)]

bins = [int(items['release_year'].min())] + bins

items['year_bin'] = pd.cut(items['release_year'],
                           bins=bins, include_lowest=True)

In [8]:
Columns.Datetime = 'last_watch_dt'

In [9]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

# Train/Test split

In [10]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [11]:
max_date = interactions[Columns.Datetime].max()

In [12]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [13]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [14]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [15]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [16]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Добавляем аватаров в обучающую выборку

1. Фанат фильмов с Киану Ривзом, смотрит фильмы и рекламу, только с его участием!
2. Человек-Патриот, смотрит фильмы только made in Russia. Считает что раньше было лучше!
3. Девочка, смотрит только мультфильмы про принцесс и ждёт своего принца на белом коне.

In [17]:
kianu_fanboy = items[(items.actors.isna() == False) & (items.actors.str.contains('Киану Ривз'))][:10]
patriot = items[(items.countries.isna() == False) & (items.countries.str.contains('Россия|СССР'))][:10]
princesses = items[(items.age_rating <= 12) & (items.keywords.str.contains('принцесса|королева', case=False))][:30]

new_items = [kianu_fanboy, patriot, princesses]

In [18]:
max_user_id = train.user_id.max()
avatar_ids = list()

for user_items in new_items:
    max_user_id += 1
    for item_id in user_items.item_id:
        last_watch_dt = f'2022-{str(randint(1,12)).zfill(2)}-{str(randint(1,28)).zfill(2)}'
        if max_user_id not in avatar_ids:
            avatar_ids.append(max_user_id)
        train = train.append({
            'user_id': max_user_id,
            'item_id': item_id,
            'last_watch_dt': last_watch_dt,
            'watched_pct': randint(70, 100),
            'weight': 3
        }, ignore_index=True)

In [19]:
avatar_map = pd.DataFrame({"user_id": avatar_ids, "name": ['kianu', 'patriot', 'pricesses']})

# Обучение моделей

In [20]:
K_RECS = 10
RANDOM_STATE = 1234
NUM_THREADS = 6

In [21]:
dataset = Dataset.construct(
    interactions_df=train
)

## Подбор гиперпараметров

In [22]:
models = {}

In [30]:
def get_model(model_name, params, n_factors, epochs):
    if model_name == 'light_fm':
        model = LightFMWrapperModel(
            LightFM(
                no_components=n_factors,
                random_state=RANDOM_STATE,
                **params
            ),
            epochs=epochs,
            num_threads=NUM_THREADS,
        )
    elif model_name == 'implicit_als':
        model = ImplicitALSWrapperModel(
            model=AlternatingLeastSquares(
                factors=n_factors,
                random_state=RANDOM_STATE, 
                num_threads=NUM_THREADS,
            ),
            **params
        )
    return model

In [32]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


models_data = [
    {'name': 'light_fm', 'params': {'loss': 'logistic'}},
    {'name': 'light_fm', 'params': {'loss': 'warp'}},
    {'name': 'implicit_als', 'params': {'fit_features_together': True}},
]
n_factors_var = list(N for N in [4,8,12])

fspace = {
    'model_data': hp.choice('model_data', models_data),
    'n_factors': hp.choice('n_factors', n_factors_var)
}
def train_and_eval(params):
    model_data = params['model_data']
    n_factors = params['n_factors']
    model = get_model(model_data['name'], model_data['params'], n_factors, 10)
    metrics = {f'MAP@{K_RECS}': MAP(k=K_RECS)}

    model.fit(dataset)
    recos = model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=K_RECS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    suffix = '__'.join([f'{k}_{v}' for k, v in model_data["params"].items()])
    models[f'{model_data["name"]}__{suffix}__n_{n_factors}'] = model

    return {'loss': -metric_values[f'MAP@{K_RECS}'], 'status': STATUS_OK}


trials = Trials()

best = fmin(fn=train_and_eval, space=fspace, algo=tpe.suggest, max_evals=6, trials=trials)

100%|██████████| 6/6 [08:05<00:00, 80.90s/trial, best loss: -0.07812375857324984] 


In [33]:
model_data = models_data[best['model_data']]
suffix = '__'.join([f'{k}_{v}' for k, v in model_data["params"].items()])
model_key = f'{model_data["name"]}__{suffix}__n_{n_factors_var[best["n_factors"]]}'
print(model_key)

light_fm__loss_warp__n_8


In [35]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in [1, 5, 10]:
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [36]:
recos = models[model_key].recommend(
    users=test[Columns.User].unique(),
    dataset=dataset,
    k=K_RECS,
    filter_viewed=True,
)
calc_metrics(metrics, recos, test, train)

{'Precision@1': 0.08252122113350505,
 'Recall@1': 0.04162495491071055,
 'Precision@5': 0.05147773440611805,
 'Recall@5': 0.12171727366166285,
 'Precision@10': 0.035410747320251026,
 'Recall@10': 0.1610629953279049,
 'MAP@1': 0.04162495491071055,
 'MAP@5': 0.07185633960397003,
 'MAP@10': 0.07812375857324984}

# Оценка моделей на аватарах

In [ ]:
recoms = models[model_key].recommend(
    users=avatar_ids,
    dataset=dataset,
    k=K_RECS,
    filter_viewed=True,
)
recoms = pd.merge(recoms, items, on='item_id')
recoms = pd.merge(recoms, avatar_map, on='user_id')[['name', 'countries', 'title', 'genres', 'age_rating', 'actors', 'keywords']]


## Фанат Киану Ривза
Модель плохо справилась, в плане предсказания фильмов с данным актёром, однако в целом попадает в в жанры, в которых он снимался

In [ ]:
recoms[:10]

,name,countries,title,genres,age_rating,actors,keywords
0,kianu,"Великобритания, США",Гнев человеческий,"боевики, триллеры",18.0,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...","ограбление, криминальный авторитет, месть, пер..."
1,kianu,Россия,Хрустальный,"триллеры, детективы",18.0,"Антон Васильев, Николай Шрайбер, Екатерина Оль...","хруст, хрусталь, хруста, хрус, полицейский, пе..."
2,kianu,Россия,Девятаев,"драмы, военные, приключения",12.0,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...","Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
3,kianu,Россия,Клиника счастья,"драмы, мелодрамы",18.0,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Клиника счастья, Клиника, Счастье, Клиника сча..."
4,kianu,Россия,Прабабушка легкого поведения,комедии,16.0,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...",", 2021, россия, прабабушка, легкого, поведения"
5,kianu,"США, Франция",Ford против Ferrari,драмы,16.0,"Кристиан Бэйл, Мэтт Дэймон, Катрина Балф, Трэй...","по мотивам романа или книги, биография, спорт,..."
6,kianu,Россия,Секреты семейной жизни,комедии,18.0,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...","брызги крови, кровь, жестокое обращение с живо..."
7,kianu,Австралия,2067: Петля времени,"драмы, фантастика",16.0,"Аарон Гленнэйн, Коди Смит-МакФи, Райан Квантен...","изменение климата, путешествие во времени, выр..."
8,kianu,США,Веном,"популярное, фантастика, триллеры, боевики, ужасы",16.0,"Том Харди, Вуди Харрельсон, Уэйд Уильямс, Мише...","Сан-Франциско, Калифорния, космический корабль..."
9,kianu,США,Мстители: Финал,"боевики, драмы, фантастика",16.0,"Роберт Дауни мл., Крис Эванс, Марк Руффало, Кр...","космическое путешествие, путешествие во времен..."


## Человек-патриот
Все фильмы, кроме одного из России, следовательно модель справилась достаточно хорошо

In [ ]:
recoms[10:20]

,name,countries,title,genres,age_rating,actors,keywords
10,patriot,"Великобритания, США",Гнев человеческий,"боевики, триллеры",18.0,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...","ограбление, криминальный авторитет, месть, пер..."
11,patriot,Россия,Хрустальный,"триллеры, детективы",18.0,"Антон Васильев, Николай Шрайбер, Екатерина Оль...","хруст, хрусталь, хруста, хрус, полицейский, пе..."
12,patriot,Россия,Девятаев,"драмы, военные, приключения",12.0,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...","Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
13,patriot,Россия,Клиника счастья,"драмы, мелодрамы",18.0,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Клиника счастья, Клиника, Счастье, Клиника сча..."
14,patriot,Россия,Прабабушка легкого поведения,комедии,16.0,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...",", 2021, россия, прабабушка, легкого, поведения"
15,patriot,Россия,Секреты семейной жизни,комедии,18.0,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...","брызги крови, кровь, жестокое обращение с живо..."
16,patriot,Россия,Подслушано,"драмы, триллеры",16.0,"Александр Hовиков, Валентина Ляпина, Никита Па...","подслушано, подслушано в контакте, социальная ..."
17,patriot,Россия,Немцы,драмы,16.0,"Евгений Коряковский, Анна Завтур, Антон Василь...","немцы, немец, про немцев, по мотивам романа, п..."
18,patriot,Россия,Афера,комедии,18.0,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Афера, Аферисты, Карантин, Пандемия, Карантин ..."
19,patriot,Россия,Сахаров. Две жизни,документальное,16.0,"Алексей Усольцев, Чулпан Хаматова, Агата Супер","Сахаров, Сахарок, Сахар, Сахар сахаров, Сахаро..."


## Ребенок, смотрящий мультики про принцесс
В рекомендациях только мультики для детей, так что в "ребёнка" попали, однако из всего списка только 1 мультфильм про принцесс.

In [ ]:
recoms[20:30]

,name,countries,title,genres,age_rating,actors,keywords
20,pricesses,США,Зверополис,"приключения, мультфильм, детективы, комедии",6.0,"Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...","аллегория, лев, бегемот, лиса, слон, овца, бел..."
21,pricesses,США,Ральф против Интернета,"мультфильм, приключения, фантастика, семейное,...",6.0,"Джон Си Райли, Сара Силверман, Галь Гадот, Тар...","видеоигра, мультфильм, продолжение, интернет, ..."
22,pricesses,США,Холодное сердце II,"фэнтези, мультфильм, музыкальные",6.0,"Идина Мензел, Кристен Белл, Джонатан Грофф, Дж...","королева, магия, королевство, плотина, дух, же..."
23,pricesses,США,Монстры на каникулах 3: Море зовёт,"мультфильм, фэнтези, приключения, комедии",6.0,"Энди Сэмберг, Селена Гомес, Кевин Джеймс, Стив...","третья часть, круизное судно, персонаж Дракула..."
24,pricesses,США,Моана,"мультфильм, фэнтези, мюзиклы",6.0,"Аулии Кравальо, Дуэйн Джонсон, Рэйчел Хаус, Те...","океан, парусник, море, мифология, остров, мюзи..."
25,pricesses,"Австралия, Бельгия",100% волк,"мультфильм, приключения, семейное, фэнтези, ко...",6.0,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...","пудель, подростковая тревога, оборотень, приня..."
26,pricesses,США,Головоломка,"фантастика, мультфильм, комедии",6.0,"Эми Полер, Филлис Смит, Ричард Кайнд, Билл Хей...","мечта, мультфильм, воображаемый друг, начальна..."
27,pricesses,США,Вперёд,"для детей, приключения, семейное, фэнтези, ком...",6.0,"Том Холланд, Крис Пратт, Джулия Луис-Дрейфус, ...","эльфы, мир фантазий, эльф, главный герой подро..."
28,pricesses,США,История игрушек 4,"мультфильм, фэнтези, комедии",6.0,"Том Хэнкс, Тим Аллен, Энни Поттс, Тони Хейл, К...","игрушка, дружба, ковбой, история игрушек 4, , ..."
29,pricesses,США,Суперсемейка 2,"фантастика, мультфильм, приключения",6.0,"Крэйг Т. Нельсон, Холли Хантер, Сара Вауэлл, Х...","семейная пара, продолжение, супергерой, дети, ..."
